In [1]:
from google.colab import drive
import sys
import os

# Montar Google Drive
drive.mount('/content/drive')

# Definir rutas base y repo
BASE_PATH = '/content/drive/MyDrive/Tesis'
REPO_NAME = 'Tesis_NN'
REPO_PATH = os.path.join(BASE_PATH, REPO_NAME)

# Agregar repo al path de Python para imports
if REPO_PATH not in sys.path:
    sys.path.append(REPO_PATH)

# Cambiar directorio actual al repo (para comandos git)
os.chdir(REPO_PATH)

print(f"Drive montado. Carpeta repo seteada en: {REPO_PATH}")


Mounted at /content/drive
Drive montado. Carpeta repo seteada en: /content/drive/MyDrive/Tesis/Tesis_NN


In [2]:
# ───────────────────────────────────────────────────────────────────────────────
# 📦 Standard Library Imports
# ───────────────────────────────────────────────────────────────────────────────
import os
from pathlib import Path    # For file and directory manipulation
import sys                  # To modify Python path for custom module imports
import csv                  # To handle CSV file reading/writing
import random               # For generating random numbers
import numpy as np          # Numerical operations and array handling
import pandas as pd         # DataFrame handling for structured data
import matplotlib.pyplot as plt  # Plotting and visualization
import time


# ───────────────────────────────────────────────────────────────────────────────
# 🌍 Third-Party Library Imports
# ───────────────────────────────────────────────────────────────────────────────
import torch                # PyTorch: deep learning framework
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms.functional as functional_transforms

# ───────────────────────────────────────────────────────────────────────────────
# 🧩 Custom Project Modules
# ───────────────────────────────────────────────────────────────────────────────

# Add custom project folder to system path to enable local module imports
sys.path.append('C:/Users/nnobi/Desktop/FIUBA/Tesis/Project')

# Import common training routines
from project_package.utils import train_common_routines as tcr

# Import model
from project_package.conv_net.ConvNet_model import SRCNN

# Import dataset manager
from project_package.dataset_manager.tensor_images_dataset import Tensor_images_dataset

# Import Sentinel-2 to Venus preprocessing utilities
from project_package.data_processing import sen2venus_routines as s2v

# Import general utility functions
from project_package.utils import utils as utils


In [3]:
#Select GPUs is they are available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ",device)

Device:  cuda


In [4]:
#Dataset
dataset = 'my_dataset2'

#Parameters for training
patched_images = 'no'
model_selection='large'
epochs = 200 # number of epochs to train the SRCNN model for
lr = 0.00001 # the learning rate
batch_size = 32
patch_size=64
stride=48
scale_value=10000

#Split porcentages for training, validation and testing
train_data_ratio=0.95
validation_data_ratio=0.04
test_data_ratio= 1 - train_data_ratio - validation_data_ratio


#Training data folder generated by load_files_tensor_data. It is in .pt files ready to be loaded
data_folder = 'C:/Users/nnobi/Desktop/FIUBA/Tesis/Project/datasets/' + dataset
data_folder = os.path.join(REPO_PATH,'datasets',dataset)
#Folder where results will be save
results_folder = os.path.join(REPO_PATH,'Conv_Net')
# Crear carpeta de resultados si no existe
os.makedirs(results_folder, exist_ok=True)

# Files to be saved
# Archivos a guardar
loss_png_file = os.path.join(results_folder, f"loss_lr={lr}_batch_size={batch_size}_model={model_selection}_patched_images={patched_images}.png")
psnr_png_file = os.path.join(results_folder, f"psnr_lr={lr}_batch_size={batch_size}_model={model_selection}_patched_images={patched_images}.png")
final_model_pth_file = os.path.join(results_folder, f"model_lr={lr}_batch_size={batch_size}_model={model_selection}_patched_images={patched_images}.pth")
file_training_losses = os.path.join(results_folder, f"training_losses_lr={lr}_batch_size={batch_size}_model={model_selection}_patched_images={patched_images}.csv")


In [6]:
#Model initialization
model = SRCNN(model_selection).to(device)
print("The model: ")
print(model)
# Get parameter count
model.count_parameters()
print(f"Total Parameters: {model.total_params:,}")
print(f"Trainable Parameters: {model.trainable_params:,}")
#Load model on multiple GPUs if available
model=tcr.multi_GPU_training(model)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)


if patched_images=="no":
    file_path_low_res = os.path.join(data_folder,'10m.pt')
    file_path_high_res= os.path.join(data_folder,'05m.pt')

    #Initialize dataset for training
    dataset = Tensor_images_dataset(file_path_low_res,file_path_high_res)

    #Split data
    train_data, val_data, test_data = tcr.data_split(dataset,train_data_ratio, validation_data_ratio, test_data_ratio)

    # Create DataLoader (ensures batch-wise loading)

    dataloader_train = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    dataloader_val = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    dataloader_test = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    print("Training with unpatched images")
elif patched_images=="yes":
    #Create DataLoader for train data
    file_path_low_res_train=data_folder/f'train_data_low_res_patched_patch_size={patch_size}_stride={stride}.pt'
    file_path_high_res_train=data_folder/f'train_data_high_res_patched_patch_size={patch_size}_stride={stride}.pt'
    dataset_train = Tensor_images_dataset(file_path_low_res_train,file_path_high_res_train)
    dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

    #Create DataLoader for validation data
    file_path_low_res_val=data_folder/f'val_data_low_res_patched_patch_size={patch_size}_stride={stride}.pt'
    file_path_high_res_val=data_folder/f'val_data_high_res_patched_patch_size={patch_size}_stride={stride}.pt'
    dataset_val = Tensor_images_dataset(file_path_low_res_val,file_path_high_res_val)
    dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

    print("Training with patched images")


The model: 
SRCNN(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), padding_mode=replicate)
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
  (relu2): ReLU(inplace=True)
  (conv3): Conv2d(128, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
  (relu3): ReLU(inplace=True)
  (conv4): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=replicate)
)
Total Parameters: 427,139
Trainable Parameters: 427,139
Model training to be done in only one GPU!
Number of training samples: 6842
Number of validation samples: 288
Number of testing samples: 72
Training with unpatched images


In [7]:
#Begin training
train_loss, val_loss = [], []
train_psnr, val_psnr = [], []
start = time.time()
for epoch in range(epochs):
    print(f"Epoch {epoch + 1} of {epochs}")
    train_epoch_loss, train_epoch_psnr = tcr.train(model, dataloader_train, optimizer, tcr.compute_loss_MSE, device)
    val_epoch_loss, val_epoch_psnr = tcr.validate(model, dataloader_val, epoch, tcr.compute_loss_MSE, device)
    print(f"Train PSNR: {train_epoch_psnr:.3f}")
    print(f"Val PSNR: {val_epoch_psnr:.3f}")
    train_loss.append(train_epoch_loss)
    train_psnr.append(train_epoch_psnr)
    val_loss.append(val_epoch_loss)
    val_psnr.append(val_epoch_psnr)
    if epoch % 5 == 0:
        tcr.save_checkpoint(epoch, model, optimizer, train_loss, filename=results_folder/f"checkpoint_epoch_{epoch}_lr={lr}_batch_size={batch_size}_model={model_selection}_patched_images={patched_images}.pth")
    with open(file_training_losses, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([train_epoch_loss, train_epoch_psnr, val_epoch_loss, val_epoch_psnr])
end = time.time()
print(f"Finished training in: {((end-start)/60):.3f} minutes")



#Plots training
plt.figure(figsize=(10, 7))
plt.plot(10*np.log10(train_loss), color='orange', label='train loss')
plt.plot(10*np.log10(val_loss), color='red', label='validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
if os.path.exists(loss_png_file):
    os.remove(loss_png_file)
plt.savefig(loss_png_file)
plt.show()
# psnr plots
plt.figure(figsize=(10, 7))
plt.plot(train_psnr, color='green', label='train PSNR dB')
plt.plot(val_psnr, color='blue', label='validation PSNR dB')
plt.xlabel('Epochs')
plt.ylabel('PSNR (dB)')
plt.legend()
if os.path.exists(psnr_png_file):
    os.remove(psnr_png_file)
plt.savefig(psnr_png_file)
plt.show()

#Save the model to disk


print('Saving model...')
if os.path.exists(final_model_pth_file):
    os.remove(final_model_pth_file)
# If using DataParallel, remove it before saving
model = model.module if hasattr(model, "module") else model
torch.save(model.state_dict(), final_model_pth_file)


Epoch 1 of 200


  0%|          | 0/214 [00:00<?, ?it/s]


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 50, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataset.py", line 420, in __getitems__
    return [self.dataset[self.indices[idx]] for idx in indices]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataset.py", line 420, in <listcomp>
    return [self.dataset[self.indices[idx]] for idx in indices]
            ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
  File "/conten